In [168]:
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
import pandas as pd

from matplotlib.font_manager import FontManager
import subprocess

fm = FontManager()
mat_fonts = set(f.name for f in fm.ttflist)

output = subprocess.check_output(
    'fc-list :lang=zh -f "%{family}\n"', shell=True)
output = output.decode('utf-8')
# print '*' * 10, '系统可用的中文字体', '*' * 10
# print output
zh_fonts = set(f.split(',', 1)[0] for f in output.split('\n'))
available = mat_fonts & zh_fonts

print('*' * 10, '可用的字体', '*' * 10)
for f in available:
    print(f)

********** 可用的字体 **********
YaHei Consolas Hybrid


In [169]:
from pandas import Series,DataFrame
import sys
sys.path.append('/home/finance-datascience/CJJ/venv/lib/python3.5/site-packages')

In [170]:
n_col = 3
n_row = 3

card_w, card_h = 496, 700

point_mapping = {
    0.5:'0',
    1:'1',
    1.5:'2',
    2:'3',    
    3:'4',
    4:'5',
    5:'5',
    6:'5'    
}

In [171]:
def generate_loc_info(df, n_col, n_row):
    df['i'] = range(len(df))
    df['row'] = (df['i']/n_col).astype(int)
    df['col'] = df['i'].mod(n_col).astype(int)
    df['col_reverse'] = (n_col-1-df['col']).astype(int)
    return df.set_index('i')

def load_image(point, point_mapping):    
    lv = point_mapping[point]
    image = Image.open('lv{}.png'.format(lv))
    image = image.resize((card_w, card_h))
    return image

def generate_canvas(card_w, card_h, n_col, n_row):
    canvas_w, canvas_h = card_w*n_col, card_h*n_row

    return Image.new('RGB', [canvas_w, canvas_h], color=(255, 255,255))

def load_description(title, items, title_font = 40, item_font = 25, title_y = 80, item_y = 150, n_char = 30, left_margin = 60, line_span = 5):
    img = Image.open("frame.jpg",).resize((card_w, card_h))
    draw = ImageDraw.Draw(img)

    title_font = ImageFont.truetype('YaHei Consolas Hybrid 1.12 For Powerline.ttf', size= title_font)

    title_width, title_height = title_font.getsize(title)

    draw.text(((card_w-title_width)/2, title_y), title, font=title_font, fill=0)

    item_font = ImageFont.truetype('YaHei Consolas Hybrid 1.12 For Powerline.ttf', size= item_font)

    y_text = item_y
    def draw_item(draw, item, y_text, n_char):
        import textwrap
        lines = textwrap.wrap(item, width=n_char)

        for i, line in enumerate(lines):
            if i>0:
                line = '   '+line
            width, height = item_font.getsize(line)
            draw.text((left_margin, y_text), line, font=item_font, fill=0)
            y_text += height

        return y_text+line_span

    for item in items:
        y_text = draw_item(draw, item, y_text, n_char)
        
    return img


In [172]:
task_df = pd.read_excel('tasks.xlsx')
task_df = task_df[task_df['printed'].isnull()]

p = 0

while True:
    canvas_front = generate_canvas(card_w, card_h, n_col, n_row)
    canvas_back = generate_canvas(card_w, card_h, n_col, n_row)    
    loc_start = p*9
    end_start = min((p+1)*9, len(task_df)-1)
    
    sub_task_df = task_df.iloc[loc_start:end_start]
    sub_task_df = generate_loc_info(sub_task_df, n_col, n_row)    

    for i, s in sub_task_df.iterrows():
        row, col = int(s['row']), int(s['col'])

        point = s['point']
        image = load_image(point, point_mapping)
        canvas_front.paste(image, [col*card_w, row*card_h, (col+1)*card_w, (row+1)*card_h])
        
    for i, s in sub_task_df.iterrows():
        row, col = int(s['row']), int(s['col_reverse'])
        title = s['title']

        items = s['description'].split('\n')
        image = load_description(title, items)
        canvas_back.paste(image, [col*card_w, row*card_h, (col+1)*card_w, (row+1)*card_h])    
   
    canvas_front.save('output/front_{}.png'.format(p))
    canvas_back.save('output/back_{}.png'.format(p))    
    
    if p+9>=len(task_df):
        break
    
    p+=1    

/home/junjiecai/venv/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/junjiecai/venv/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/junjiecai/venv/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in